In [199]:
from collections import ChainMap
import pandas as pd
import numpy as np

In [200]:
data = pd.read_csv('solar_panel_assembly_expanded_dataset.csv', encoding='utf-8')

In [201]:
df = pd.DataFrame(data)

In [202]:
df.columns

Index(['Date', 'Shift', 'Panel Serial No.', 'Panel Type', 'Assembly Line',
       'Cycle Time (s)', 'Number of Cells', 'Cell Alignment Deviation (mm)',
       'Glass Thickness (mm)', 'Junction Box Attached', 'Power Output (W)',
       'Efficiency (%)', 'Insulation Resistance (MΩ)', 'Flash Test Result',
       'Visual Inspection', 'Operator ID', 'Final Inspection'],
      dtype='object')

In [203]:
df.head(5)

,Date,Shift,Panel Serial No.,Panel Type,Assembly Line,Cycle Time (s),Number of Cells,Cell Alignment Deviation (mm),Glass Thickness (mm),Junction Box Attached,Power Output (W),Efficiency (%),Insulation Resistance (MΩ),Flash Test Result,Visual Inspection,Operator ID,Final Inspection
0,2025-04-21,Shift 1,SP819428,Polycrystalline,Line 1,1005.9,72,0.17,3.9,No,358.3,18.03,604.6,Fail,Crack,EMP193,Pass
1,2025-04-08,Shift 1,SP736702,Thin-Film,Line 3,980.8,60,0.24,3.1,No,312.5,19.62,990.1,Pass,Scratch,EMP193,Pass
2,2025-04-04,Shift 1,SP352954,Polycrystalline,Line 3,887.4,60,0.37,3.3,No,383.7,17.21,227.8,Pass,Crack,EMP193,Pass
3,2025-04-17,Shift 2,SP725540,Polycrystalline,Line 1,760.8,72,0.14,3.3,Yes,350.4,20.30,619.7,Pass,Scratch,EMP193,Pass
4,2025-04-07,Shift 1,SP189001,Monocrystalline,Line 3,1098.1,72,0.27,3.7,Yes,283.9,21.30,707.5,Fail,Pass,EMP193,Pass


# ChainMap-Based Exercises Using the Dataset

### **Exercise 1: Effective Assembly Configuration**

**Goal:**
For each unique `Panel Type`, create a fallback configuration using three layers:

1. `live_settings` (incomplete)
2. `cached_settings` (older complete config)
3. `defaults` (known good values)

**Problem:**
Simulate retrieving the effective configuration for `Polycrystalline` panels, including:

* `Glass Thickness (mm)`
* `Number of Cells`
* `Cell Alignment Deviation (mm)`

**Expected Output Example:**

```
Effective Polycrystalline Config ➜ {'Glass Thickness (mm)': 3.3, 'Number of Cells': 72, 'Cell Alignment Deviation (mm)': 0.27}
```

In [204]:
# filter rows for `Polycrystalline` panels
polycrystalline_df = df[df['Panel Type'] == 'Polycrystalline']

In [205]:
polycrystalline_df.head(3)

,Date,Shift,Panel Serial No.,Panel Type,Assembly Line,Cycle Time (s),Number of Cells,Cell Alignment Deviation (mm),Glass Thickness (mm),Junction Box Attached,Power Output (W),Efficiency (%),Insulation Resistance (MΩ),Flash Test Result,Visual Inspection,Operator ID,Final Inspection
0,2025-04-21,Shift 1,SP819428,Polycrystalline,Line 1,1005.9,72,0.17,3.9,No,358.3,18.03,604.6,Fail,Crack,EMP193,Pass
2,2025-04-04,Shift 1,SP352954,Polycrystalline,Line 3,887.4,60,0.37,3.3,No,383.7,17.21,227.8,Pass,Crack,EMP193,Pass
3,2025-04-17,Shift 2,SP725540,Polycrystalline,Line 1,760.8,72,0.14,3.3,Yes,350.4,20.30,619.7,Pass,Scratch,EMP193,Pass


In [206]:
# simulate 3 layers
# 1. live_settings: Assume `Number of Cells` is missing
live_settings = {
    'Glass Thickness (mm)': polycrystalline_df.iloc[0]['Glass Thickness (mm)'],
    'Cell Alignment Deviation (mm)': polycrystalline_df.iloc[0]['Cell Alignment Deviation (mm)'],
}

In [207]:
live_settings

{'Glass Thickness (mm)': np.float64(3.9),
 'Cell Alignment Deviation (mm)': np.float64(0.17)}

In [208]:
# 2. cached_settings: Previous config, also incomplete
cached_settings = {
    'Number of Cells': polycrystalline_df.iloc[1]['Number of Cells'],
    'Glass Thickness (mm)': polycrystalline_df.iloc[1]['Glass Thickness (mm)'],
}

In [209]:
cached_settings

{'Number of Cells': np.int64(60), 'Glass Thickness (mm)': np.float64(3.3)}

In [210]:
# defaults: fully populated known good values
defaults = {
    'Glass Thickness (mm)': 3.3,
    'Number of Cells': 72,
    'Cell Alignment Deviation (mm)': 0.27,
}

In [211]:
effective_config = ChainMap(live_settings, cached_settings, defaults)

In [212]:
# extract only 3 needed keys
final_config = {
    'Glass Thickness (mm)': effective_config['Glass Thickness (mm)'],
    'Number of Cells': effective_config['Number of Cells'],
    'Cell Alignment Deviation (mm)': effective_config['Cell Alignment Deviation (mm)'],
}

In [213]:
final_config

{'Glass Thickness (mm)': np.float64(3.9),
 'Number of Cells': np.int64(60),
 'Cell Alignment Deviation (mm)': np.float64(0.17)}

### **Exercise 2: Flash Test Override by Operator**

**Goal:**
Allow an operator to override `Flash Test Result` temporarily during QA review.

**Problem:**
Simulate a ChainMap where:

* `qa_override = {'Flash Test Result': 'Pass'}`
* It overrides the result for a specific serial number (`SP819428`)

**Expected Output:**

```
SP819428 ➜ Flash Test Result (QA): Pass
```

In [214]:
record = df[df['Panel Serial No.'] == 'SP819428'].iloc[0]

In [215]:
record['Flash Test Result']

'Fail'

In [216]:
qa_override = {
    'Flash Test Result': 'Pass',
}

In [217]:
original_layer = {
    'Flash Test Result': record['Flash Test Result'],
}

In [218]:
qa_result = ChainMap(qa_override, original_layer)

In [219]:
effective_flash_test_result = {
    'serial number': record['Panel Serial No.'],
    'result': qa_result['Flash Test Result'],
}

In [220]:
effective_flash_test_result

{'serial number': 'SP819428', 'result': 'Pass'}

In [221]:
print(f'{effective_flash_test_result["serial number"]} --> Flash Test Result (QA): {effective_flash_test_result["result"]}')

SP819428 --> Flash Test Result (QA): Pass


### **Exercise 3: Fallback for Missing Resistance Values**

**Goal:**
Use ChainMap to fill in missing `Insulation Resistance (MΩ)` values using:

* live → calculated average → engineering default

**Problem:**
If a panel’s resistance is missing, try getting it from an operator’s average, or fallback to a default of `500 MΩ`.

**Expected Output:**

```
Fallback Resistance ➜ 500.0
```

In [222]:
# Simulate a panel with missing `Insulation Resistance (MΩ)`
sample_row = df.iloc[0].copy()

In [223]:
sample_row

Date                                  2025-04-21
Shift                                    Shift 1
Panel Serial No.                        SP819428
Panel Type                       Polycrystalline
Assembly Line                             Line 1
Cycle Time (s)                            1005.9
Number of Cells                               72
Cell Alignment Deviation (mm)               0.17
Glass Thickness (mm)                         3.9
Junction Box Attached                         No
Power Output (W)                           358.3
Efficiency (%)                             18.03
Insulation Resistance (MΩ)                 604.6
Flash Test Result                           Fail
Visual Inspection                          Crack
Operator ID                               EMP193
Final Inspection                            Pass
Name: 0, dtype: object

In [224]:
sample_row['Insulation Resistance (MΩ)'] = np.nan

In [225]:
sample_row.tail(5)

Insulation Resistance (MΩ)       NaN
Flash Test Result               Fail
Visual Inspection              Crack
Operator ID                   EMP193
Final Inspection                Pass
Name: 0, dtype: object

In [226]:
operator_id = sample_row['Operator ID']

In [227]:
# compute operator average if available
operator_panels = df[
    (df['Operator ID'] == operator_id) &
    (df['Insulation Resistance (MΩ)'].notna())
]

In [228]:
operator_avg = operator_panels['Insulation Resistance (MΩ)'].mean()

In [229]:
operator_avg

np.float64(555.6684210526315)

In [230]:
# create fallback layers
live_layer = {}
average_layer = {}

In [231]:
if not pd.isna(sample_row['Insulation Resistance (MΩ)']):
    live_layer['Insulation Resistance (MΩ)'] = sample_row['Insulation Resistance (MΩ)']
if not pd.isna(operator_avg):
    average_layer['Insulation Resistance (MΩ)'] = operator_avg

In [232]:
default_layer = {
    'Insulation Resistance (MΩ)': 500
}

In [233]:
resistance_chain = ChainMap(live_layer, average_layer, default_layer)

In [234]:
resolved_resistance = resistance_chain['Insulation Resistance (MΩ)']

In [235]:
resolved_resistance

np.float64(555.6684210526315)

### **Exercise 4: Junction Box Status Resolution**

**Goal:**
Resolve whether a panel has a junction box attached using 3 fallback sources:

1. Line data (`Line 1` logs)
2. Panel type heuristic
3. Default: `No`

**Problem:**
For panels with unknown attachment, simulate resolution using ChainMap.

**Expected Output:**

```
Panel SP736702 ➜ Junction Box Attached: No
```

In [236]:
sample_panel = df[df['Panel Serial No.'] == 'SP736702'].iloc[0]

In [237]:
sample_panel

Date                             2025-04-08
Shift                               Shift 1
Panel Serial No.                   SP736702
Panel Type                        Thin-Film
Assembly Line                        Line 3
Cycle Time (s)                        980.8
Number of Cells                          60
Cell Alignment Deviation (mm)          0.24
Glass Thickness (mm)                    3.1
Junction Box Attached                    No
Power Output (W)                      312.5
Efficiency (%)                        19.62
Insulation Resistance (MΩ)            990.1
Flash Test Result                      Pass
Visual Inspection                   Scratch
Operator ID                          EMP193
Final Inspection                       Pass
Name: 1, dtype: object

In [238]:
sample_panel['Junction Box Attached'] = np.nan

In [239]:
sample_panel

Date                             2025-04-08
Shift                               Shift 1
Panel Serial No.                   SP736702
Panel Type                        Thin-Film
Assembly Line                        Line 3
Cycle Time (s)                        980.8
Number of Cells                          60
Cell Alignment Deviation (mm)          0.24
Glass Thickness (mm)                    3.1
Junction Box Attached                   NaN
Power Output (W)                      312.5
Efficiency (%)                        19.62
Insulation Resistance (MΩ)            990.1
Flash Test Result                      Pass
Visual Inspection                   Scratch
Operator ID                          EMP193
Final Inspection                       Pass
Name: 1, dtype: object

In [245]:
line_data_layer = {}
if pd.isna(sample_panel['Junction Box Attached']):
    line_data_layer['Serial Number'] = sample_panel['Panel Serial No.']

In [246]:
panel_type_heuristic = {
    'Serial Number': sample_panel['Panel Serial No.'],
}

In [247]:
default = {
    'Junction Box Attached': 'No',
}

In [248]:
resolution_jb = ChainMap(line_data_layer, panel_type_heuristic, default)

In [249]:
# Panel SP736702 ➜ Junction Box Attached: No
print(f'Panel {resolution_jb["Serial Number"]} --> Junction Box Attached: {resolution_jb["Junction Box Attached"]}')

Panel SP736702 --> Junction Box Attached: No


### 🔹**Exercise 5: Cell Count Resolution**

**Goal:**
If `Number of Cells` is missing or inconsistent, fallback to:

1. Panel Type average
2. Line average
3. Default: 60

**Expected Output:**

```
Resolved Cell Count ➜ 72
```

---

### 🔹**Exercise 6: Determine True Final Status**

**Goal:**
Combine `Flash Test Result`, `Visual Inspection`, and `Final Inspection` using override priority:

1. QA override (ChainMap layer 1)
2. Inspector override
3. Original result

**Expected Output:**

```
Final Result: FAIL (due to visual inspection)
```

---

### 🔹**Exercise 7: Fallback Efficiency Calculation**

**Goal:**
If `Efficiency (%)` is missing, resolve it via:

1. Flash Pass average
2. Line-specific average
3. Global average

**Expected Output:**

```
Resolved Efficiency: 18.77%
```

---

### 🔹**Exercise 8: Override Operator ID for Batch**

**Goal:**
For a specific batch of panels, apply temporary operator override using ChainMap:

* Layer 1: Temp override → `{'Operator ID': 'EMP999'}`
* Layer 2: Original records

**Expected Output:**

```
Effective Operator ID for SP819428 ➜ EMP999
```

---

### 🔹**Exercise 9: Filter Panels with Resolved Power Output**

**Goal:**
Use ChainMap to handle panels with:

* Missing power output → fallback to type-based average or 300 W

**Expected Output Example:**

```
Panel SPxxxxxx ➜ Power Output: 312.5 W (fallback used)
```

---

### 🔹**Exercise 10: Combine All Config Layers Per Shift**

**Goal:**
For a given `Shift`, create a 3-layer ChainMap config:

* Shift-specific live tuning
* Assembly line fallback
* Global engineering spec

**Expected Output:**

```
Shift 1 Config ➜ {'Glass Thickness (mm)': 3.7, 'Efficiency (%)': 19.5, 'Cycle Time (s)': 980.8}
```

---

Let me know if you’d like the starter templates or to work through each problem interactively next!
